In [3]:
import pandas as pd

# Cargar el archivo CSV con los datos filtrados y con Fold Change
file_path_fc = 'deSeqRes_filtrado_Propionibacterium freudenreichii_with_FC.csv'
data_with_fc = pd.read_csv(file_path_fc)

# Renombrar las columnas
data_with_fc = data_with_fc.rename(columns={
    'GeneSymbol': 'Genes',
    'DirectionalFoldChange': 'FoldChange'
})

# Crear una nueva columna 'Regulation' basada en el signo de 'FoldChange'
data_with_fc['Regulation'] = data_with_fc['FoldChange'].apply(lambda x: 'UpRegulated' if x > 0 else 'DownRegulated')

# Tomar el valor absoluto de 'FoldChange'
data_with_fc['FoldChange'] = data_with_fc['FoldChange'].abs()

# Cargar el archivo CSV con los términos GO
file_path_go = r'C:\\Users\\3060\\OneDrive\\Documentos\\Tesis de Grado\\Protocolo\\4_Análisis de Enriquecimiento Funcional\\Propionibacterium freudenreichii\\Enrichr\\Ontologies\\GO_Biological_Process_2023_table.txt'
df = pd.read_csv(file_path_go, sep='\t')

# Eliminar las columnas especificadas
columns_to_drop = ["Overlap", "P-value", "Old P-value", "Old Adjusted P-value", "Odds Ratio", "Combined Score"]
df = df.drop(columns=columns_to_drop)

# Filtrar los registros con "Adjusted P-value" <= 0.05
df_filtered = df[df["Adjusted P-value"] <= 0.05]

# Expandir la columna "Genes" para que cada gen tenga su propia fila
df_expanded = df_filtered.assign(Genes=df_filtered['Genes'].str.split(';')).explode('Genes')

# Agrupar los términos asociados a cada gen en una lista
df_grouped = df_expanded.groupby('Genes').agg({
    'Term': lambda x: list(x),
    'Adjusted P-value': lambda x: list(x)  # Mantener todos los valores de "Adjusted P-value" para cada gen
}).reset_index()

# Combinar df_grouped con data_with_fc
df_combined = pd.merge(df_grouped, data_with_fc, on='Genes', how='left')

# Ordenar los registros por el valor de "FoldChange" del más alto al más bajo
df_combined_sorted = df_combined.sort_values(by='FoldChange', ascending=False)

# Convertir las columnas 'Term' y 'Adjusted P-value' a strings con los elementos de la lista separados por comas
df_combined_sorted['Term'] = df_combined_sorted['Term'].apply(lambda x: ', '.join(x))
df_combined_sorted['Adjusted P-value'] = df_combined_sorted['Adjusted P-value'].apply(lambda x: ', '.join(map(str, x)))

# Mostrar las primeras filas del dataframe combinado y ordenado
print(df_combined_sorted.head())

# Guardar el DataFrame combinado y ordenado en un nuevo archivo CSV
df_combined_sorted.to_csv('deSeqRes_combined_with_GO_Biological_Process_terms_sorted.csv', index=False)

# Guardar el DataFrame combinado y ordenado en un archivo Excel
df_combined_sorted.to_excel('deSeqRes_combined_with_GO_Biological_Process_terms_sorted.xlsx', index=False)

     Genes                                               Term  \
10  CDKN1A  Regulation Of Cyclin-Dependent Protein Kinase ...   
12  CDKN2B  Regulation Of Cyclin-Dependent Protein Kinase ...   
1    BRSK2  Mitotic Cell Cycle Phase Transition (GO:004477...   
11  CDKN1C  Regulation Of Cyclin-Dependent Protein Kinase ...   
15     NES  Mitotic Cell Cycle Phase Transition (GO:004477...   

                                     Adjusted P-value  Log2FoldChange  \
10  0.0001516703557587678, 0.0035377614195434, 0.0...        4.205535   
12  0.0001516703557587678, 0.0035377614195434, 0.0...        2.841668   
1              0.0360141282060231, 0.0496462183350143        2.637889   
11          0.0001516703557587678, 0.0235614479185477        2.437113   
15             0.0360141282060231, 0.0496462183350143        2.020522   

    FoldChange   Regulation  
10   18.449818  UpRegulated  
12    7.168482  UpRegulated  
1     6.224201  UpRegulated  
11    5.415570  UpRegulated  
15    4.057305  UpRe